# berkeley_sda_data
This notebook uses selenium to navigate through the Berkeley Survey Documentation and Analysis (SDA) to download Networth studies from the Survey of Consumer Finances (SCF) subsite.

### Usage:
1. Manually run each cell in Step 01.  This will result in you having to respond to the download dialog box from Firefox.
2. Choose "save file", and then follow the instructions from cell Step 02.
3. Then run the rest of the cells to see the graph of the data aggregation.

### Important
This version of sel_scrape and selenium probably will not work with versions of Firefox new than version 53.  To get that version for the Mac, see the project https://github.com/bgithub1/selscrape.

In [33]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.graph_objs.layout import Font,Margin
import plotly.graph_objs as go
from plotly.offline import iplot
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import sel_scrape as sc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
import time

### Step 01: Use selenium to navigate to the sda.berkeley.edu/sdaweb/analysis website, create subset of their networth data, and to click on the button that downloads the file.
You must respond to the dialog box that the website generates to complete the download

In [34]:
SCF_HOME_PAGE='https://sda.berkeley.edu/sdaweb/analysis/?dataset=scfcomb'
DOWNLOAD_CUSTOM_SUBSET_BUTTON = '//button[@id="top-form:cb-subset"]'
CSV_VIEW_XPATH = '//input[@type="radio" and @value="CSV"]'
CODE_BOOK_XPATH = '//input[@id="subsetWizardForm:subsetTabview:codebook"]'
SELECT_VARIABLES_TAB_XPATH = '//a[contains(@href,"selectvars")]'
SELECT_YEAR_XPATH = '//li[contains(@id,"subsetTree:0")]//span[contains(@class,"ui-chkbox-icon")]'
SELECT_NETWORTH_XPATH = '//li[contains(@id,"subsetTree:10")]//span[contains(@class,"ui-chkbox-icon")]'
CREATE_FILES_TAB_XPATH = '//a[contains(@href,"createfiles")]'
SELECT_CREATE_BUTTON = '//button[contains(@id,"j_idt1614")]'
SELECT_DOWNLOAD_BUTTON = '//button[contains(@id,"j_idt1620")]'

In [35]:
sela = sc.SelScrape(headless=False)
sela.goto(SCF_HOME_PAGE)
time.sleep(1)

In [36]:
web_page_download = sela.findxpath(DOWNLOAD_CUSTOM_SUBSET_BUTTON)
if web_page_download['status'] is None:
    sela.click_element(DOWNLOAD_CUSTOM_SUBSET_BUTTON)
    time.sleep(1)
    sela.click_element(CSV_VIEW_XPATH)
    time.sleep(1)
    sela.click_element(CODE_BOOK_XPATH)
    time.sleep(1)
    sela.click_element(SELECT_VARIABLES_TAB_XPATH)
    time.sleep(2)
    sela.click_element(SELECT_YEAR_XPATH)
    time.sleep(1)
    sela.click_element(SELECT_NETWORTH_XPATH) 
    time.sleep(2)
    create_files_element = sela.findxpath(CREATE_FILES_TAB_XPATH)['value'][0]
    sela.driver.execute_script("return arguments[0].scrollIntoView(true);", create_files_element)
    sela.click_element(CREATE_FILES_TAB_XPATH)
    time.sleep(3)
    sela.click_element(SELECT_CREATE_BUTTON)
    time.sleep(5)
    sela.click_element(SELECT_DOWNLOAD_BUTTON)    
    time.sleep(5)

### Step 02: Replace ```SDA_FILE_PATH``` below with the file that you downloaded, and run the other cells

### !!!! Stop Here and navigate to the Firefox download dialog box and choose save file.  

#### Then either replace the current file in this project named 'SCFP2016.csv' with your new file - keeping the same name of SCFP2016.csv, or set the variable ```SDA_FILE_PATH = your_new_path```

In [38]:
SDA_FILE_PATH = 'SCFP2016.csv'

In [39]:


def plotly_plot(df_in,x_column,plot_title=None,
                y_left_label=None,y_right_label=None,
                bar_plot=False,figsize=(16,10),
                number_of_ticks_display=20,
                yaxis2_cols=None):
    ya2c = [] if yaxis2_cols is None else yaxis2_cols
    ycols = [c for c in df_in.columns.values if c != x_column]
    # create tdvals, which will have x axis labels
    td = list(df_in[x_column]) 
    nt = len(df_in)-1 if number_of_ticks_display > len(df_in) else number_of_ticks_display
    spacing = len(td)//nt
    tdvals = td[::spacing]
    
    # create data for graph
    data = []
    # iterate through all ycols to append to data that gets passed to go.Figure
    for ycol in ycols:
        if bar_plot:
            b = go.Bar(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        else:
            b = go.Scatter(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        data.append(b)

    # create a layout
    layout = go.Layout(
        title=plot_title,
        xaxis=dict(
            ticktext=tdvals,
            tickvals=tdvals,
            tickangle=45,
            type='category'),
        yaxis=dict(
            title='y main' if y_left_label is None else y_left_label
        ),
        yaxis2=dict(
            title='y alt' if y_right_label is None else y_right_label,
            overlaying='y',
            side='right'),
        margin=Margin(
            b=100
        )        
    )

    fig = go.Figure(data=data,layout=layout)
    return fig


ACCESS the Berkeley SDA SCFP2016 data

In [40]:
df = pd.read_csv(SDA_FILE_PATH)

select the 12th network percentile category (higest 1% of networth) and extract out the net worth column

In [41]:
df[df.NWPCTLECAT==12].NETWORTH.sum()

352035884910.2

In [42]:
sorted(df.columns.values)

['ACTBUS',
 'AGE',
 'AGECL',
 'ANNUIT',
 'ANYPEN',
 'ASSET',
 'ASSETCAT',
 'BCALL',
 'BDONT',
 'BFINPLAN',
 'BFINPRO',
 'BFRIENDWORK',
 'BINTERNET',
 'BMAGZNEWS',
 'BMAILADTV',
 'BNKRUPLAST5',
 'BOND',
 'BOTHER',
 'BPLANCJ',
 'BSELF',
 'BSHOPGRDL',
 'BSHOPMODR',
 'BSHOPNONE',
 'BUS',
 'BUSSEFARMINC',
 'BUSVEH',
 'CALL',
 'CANTMANG',
 'CASHLI',
 'CCBAL',
 'CDS',
 'CHECKING',
 'CKCONNECTN',
 'CKCONVPAYRL',
 'CKLOCATION',
 'CKLONGTIME',
 'CKLOWFEEBAL',
 'CKMANYSVCS',
 'CKOTHCHOOSE',
 'CKPERSONAL',
 'CKRECOMFRND',
 'CKSAFETY',
 'COMUTF',
 'CONSPAY',
 'CRDAPP',
 'CREDIT',
 'CURRPEN',
 'DBPLANCJ',
 'DBPLANT',
 'DCPLANCJ',
 'DEBT',
 'DEBT2INC',
 'DEQ',
 'DONTLIKE',
 'DONTWANT',
 'DONTWRIT',
 'EDCL',
 'EDN_INST',
 'EDUC',
 'EHCHKG',
 'EMERGBORR',
 'EMERGCUT',
 'EMERGPSTP',
 'EMERGSAV',
 'EQUITINC',
 'EQUITY',
 'EXPENSHILO',
 'FAMSTRUCT',
 'FARMBUS',
 'FARMBUS_KG',
 'FEARDENIAL',
 'FIN',
 'FINLIT',
 'FOODAWAY',
 'FOODDELV',
 'FOODHOME',
 'FORECLLAST5',
 'FUTPEN',
 'GBMUTF',
 'GOVTBND',
 'HASSET

In [44]:
df_agg = df[['NWPCTLECAT','NETWORTH']].groupby('NWPCTLECAT',as_index=False).sum()
iplot(plotly_plot(df_agg,x_column='NWPCTLECAT',bar_plot=True,plot_title='Networth by Networth Category (12 is the richest)'))
